<a href="https://colab.research.google.com/github/luke-scot/damage-assessment/blob/main/colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Confidence-aware belief propagation for multimodal data
This notebook runs a demonstration of the belief propagtion (BP) framework initially created for post-disaster damage assessment by Luke Cullen at the University of Cambridge. For development, please see the [Github repository](https://github.com/luke-scot/damage-assessment). Alternatively for further details, the full report accompanying this project is available [here](https://drive.google.com/file/d/1IZ3B0m5FrPybTsKMUQ52ExvheMin9WdB/view?usp=sharing).

To run each cell simply press Shift & Enter simultaneously.

## Setup
Run the first cell to download the latest version of the demo functions.

In [ ]:
%%capture
# Clone repository
! git clone https://github.com/luke-scot/damage-assessment.git
%cd damage-assessment

# Install uncommon packages
%pip install install rioxarray geopandas ipyleaflet gdown pygeos

import colab_interactions as it

## Introduction to Damage Assessment application

A brief introduction to damage assessment and belief propagation.

In the immediate aftermath of a disaster, such as the 4th August 2020 Beirut port explosion, there are 2 priority tasks for recovery efforts:
* Search & Rescue - Need to prioritise most damaged area to send reconnaissance teams, and save lives of people trapped.

* Damage Assessment - Classify buildings to prevent further casualties and assess humanitarian/reconstruction needs.

Both rely on rapid knowledge of building damages. 

This project explores using Belief Propagation to rapidly assess infrastructure damage by combining all data available into a graph representation of the affected area. I will focus mainly on remote sensing (satellite) data due to its availability and past efficiency in identifying damage. 

This model brings two new features to automating damage assessments both key to creating actionable information for responders:
*   Uncertainty quantification - Hence choice of confidence-aware NetConf algorithm as basis for BP (originally created by [Eswaran et al. (2017)](https://dhivyaeswaran.github.io/papers/sdm17-netconf.pdf).
*   Multimodality - Graph representation allows us to combine any data, however incomplete or useful, as it becomes available in a post-disaster scenario.

## What is Belief Propagation?

Before delving in to large-scale spatial applications, take a look at what BP is doing at an interpretable level. This part will show the code as we step through so you can get a taste of the magic that goes on behind close doors in the real application scenario.

Choose your parameters in the first cell and then keep going to see where you end up, nodes unassigned to either class will begin as unknowns. Leave plenty as this is how you'll get to understand what BP is doing.

In [ ]:
# Choose your graph parameters - you can always come back once you understand what these mean
nodes = 12 # Total number of nodes
zeros = 2 # Nodes in class 0 
ones = 2 # Nodes in class 1
neighbours = 2 # Edges will be created to this number of nearest neighbours
cmap = 'RdYlGn' # No need to change unless you're fancying some groovy colours today

In [ ]:
# Import those handy pre-made functions
import random
import numpy as np
import networkx as nx
import matplotlib as mpl
from netconf import netconf
import matplotlib.pyplot as plt
from sklearn.neighbors import kneighbors_graph

In [ ]:
# Create a graph and add your nodes
G = nx.Graph() # Initialise the graph - nodes have got to go somewhere
G.add_nodes_from(range(nodes)) # Nodes going on
pos = nx.spring_layout(G) # Fix the positions of the nodes
nx.draw(G, pos=pos, node_size=1500, with_labels=True) # Let's have a look

In [ ]:
"""
Good start, now let's add our prior knowledge.

We'll randomly assign which nodes are to be in classes 0 and 1, with the rest 
being ignorant - i.e. 0.5 prior belief.
"""
priors = [1]*ones+[0]*zeros+[0.5]*(nodes-(ones+zeros))
np.random.shuffle(priors)

# Let's draw it again
nx.draw(G, pos=pos, node_size=1500, with_labels=True, node_color=priors, cmap=cmap)

# Don't worry about these, it's just adding a colorbar to look pretty
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin = 0, vmax=1))
c = plt.colorbar(sm)
c.set_label('Class 1 probability', fontsize=14)

In [ ]:
"""
We have our prior beliefs! Let's get some edges involved.

kneighbours_graph is a function which finds the nearest neighbours to each
node according to it's value. The values of our nodes are 0 to n (n being how
many nodes you chose) as displayed on each graph
"""
# Get edges
values = np.array(range(nodes)).reshape(-1,1) # Get our values in a vector
edges = kneighbors_graph(values,neighbours,mode='connectivity',include_self=False)

# Just a matrix re-shuffle to make the output usable, don't panic
edges = np.array(edges.nonzero()).reshape(2,-1).transpose() 

# Let's take another look
nx.draw(G, pos=pos, node_size=1500, with_labels=True, node_color=priors, cmap=cmap, edgelist=edges)
c = plt.colorbar(sm)
c.set_label('Class 1 probability', fontsize=14)

In [ ]:
"""
Ok, here we are, let's fire up the beast!

Netconf is implemented in the netconf.py file, feel free to have a browse but there's 
not much need unless you're an equations enthusiast.
"""
priors = np.array(priors).reshape(-1,1) # Just a little matrix shimmy
posteriors, _ = netconf(edges,priors, verbose=True) # Here it goes! 

In [ ]:
"""
Well, wasn't that exhilirating!
Ok, now we'll plot up the priors against the posteriors to see what actually went 
down.
"""
fig, axs = plt.subplots(1, 2, figsize=[15,5]) # Initialise a figure 

# Let's draw the prior beliefs
nx.draw(G, pos=pos, node_size=1500, with_labels=True, node_color=priors, cmap=cmap, edgelist=edges, ax=axs[0])
axs[0].set_title('Prior beliefs', fontsize=15)
c = plt.colorbar(sm, ax=axs[0])
c.set_label('Class 1 probability', fontsize=14)

# And now the posterior beliefs
nx.draw(G, pos=pos, node_size=1500, with_labels=True, node_color=posteriors, cmap=cmap, edgelist=edges, ax=axs[1])
axs[1].set_title('Posterior beliefs', fontsize=15)
c = plt.colorbar(sm, ax=axs[1])
c.set_label('Class 1 probability', fontsize=14)

Nice Graphs! So hopefully this short BP demo has given you a glimpse into the world of graph representation and belief propagation. Now, imagine a graph a lot (and I mean A LOT) bigger where each node could represent a 50x50cm square over a city... Daunting? Well that's where we're heading next. Keep scrolling to see how BP can be applied to real-world problems with large-scale spatial data.

## Real-world application
Ok, now that you're a belief propagation expert, let's look at some slightly more interesting applications. 

The first cell will give you 3 options:
1.   Beirut damage assessment scenario - This is the target application.
2.   Houston land classification - This is a demonstration to show how the model performs on spatial data in a 'well-behaved' setting.
3.   None - Free reign, input whatever you feel like as long as you have a ground truth (shapefile or imagefile) and image data. You can start with the above defaults and adjust from there though if you don't want to be plunged in a the deep end.

I would recommend starting with the Houston demonstration (I know it's second 
on the dropdown) and once you've been through that come back up to play with the Beirut damage assessment which is the real application I was aiming for in this study.

> Colab tips - Unfortunately Colab isn't keen on interactions and doesn't support ipyleaflet, so this demo has less features, and is less pretty, than the real model on [Github](https://github.com/luke-scot/damage-assessment/blob/main/demo.ipynb) created in Descartes Labs. However, you will have no problems with installing packages which is why I've chosen Colab. If runtime becomes an issue, make sure you're not running multiple sessions.

In [ ]:
# Run me!
defaults = it.get_defaults()

In [ ]:
# Picked your default? Run me to display your inputs.
inputs = it.input_parameters(defaults)

In [ ]:
import os
import math
import imageio
import numpy as np
import geopandas as gpd
import ipywidgets as ipw
import ipyleaflet as ipl
import matplotlib.pyplot as plt
import matplotlib.colors as clrs
from branca.colormap import linear
from ipyleaflet import LayersControl
from scipy.interpolate import griddata
import folium as fl
from branca.element import Template, MacroElement

import netconf as nc
import plotting as pl
import imports as ip
import transforms as tr


  

In [ ]:
"""Please confirm your types before running this one
# If it's the first time you've run it for the application - this will retrieve
# the relevant data from Google Drive, it should be quick but bare with."""
download = it.which_download(inputs) 
! bash $download
parameters = model_parameters(inputs)

In [ ]:
# You only need to import the data once (do not need to re-run for changing class-edge-node configurations)
imports = it.import_data(parameters)

In [ ]:
# Now we'll group the labels into classes dependent on your choices
classified = it.classify_data(imports)

In [ ]:
# Ok, we are ready to rumble! Hit the button.
output = it.run_bp(classified)

In [ ]:
# It's done! Or should be, first let's look at performance metrics.
plots = it.evaluate_output(output)

In [ ]:
# Fancy saving the plot? Go for it.
it.save_plot(plots, location='results/performancePlot.png')

In [ ]:
""" Ok let's see the final classification map!
This is currently ony supported for 2-class classification.
For a fully interactive plot you'll have to bite the bullet
and get the real model as Colab is not quite upto it unfortunately"""

mapping = it.map_result(plots)

That's all folks! Scroll back up and have a play around. Or if you're feeling the BP groove, hit it up with your own data!

## Conclusion
Hopefully this demo has shown you the real potential of graph-based belief propagation. The uncertainty quantification and multimodality are two very appealing features for any application, notably in the scientific field. 
I'm always keen to hear of any new applications in graph representation learning. So, even if you didn't like my demo, please drop me a line at lshc3@cam.ac.uk, maybe you can tell me where I'm going wrong!